In [4]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, sum as _sum, abs as _abs, isnan, lit, to_timestamp
from prettytable import PrettyTable

StatementMeta(, 9873af4d-6a72-4731-8650-5ef8218963c7, 6, Finished, Available, Finished)

In [2]:
# Load Silver tables
orders= spark.read.table("olist_orders_cleaned")
items = spark.read.table("olist_items_cleaned")
customers = spark.read.table("olist_customers_cleaned")
sellers = spark.read.table("olist_sellers_cleaned")
products = spark.read.table("olist_products_cleaned")
payments = spark.read.table("olist_payments_cleaned")
reviews = spark.read.table("olist_reviews_cleaned")

StatementMeta(, 9873af4d-6a72-4731-8650-5ef8218963c7, 4, Finished, Available, Finished)

1. Schema & Data Types

In [3]:
silver_tables= {
    "orders": spark.read.table("olist_orders_cleaned"),
    "items": spark.read.table("olist_items_cleaned"),
    "customers": spark.read.table("olist_customers_cleaned"),
    "sellers": spark.read.table("olist_sellers_cleaned"),
    "products": spark.read.table("olist_products_cleaned"),
    "payments": spark.read.table("olist_payments_cleaned"),
    "reviews": spark.read.table("olist_reviews_cleaned")
}

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 5, Finished, Available, Finished)

In [4]:
def validate_column_types(df_dict):
    results = []
    
    for table_name, df in df_dict.items():
        schema = df.schema
        for field in schema:
            colname = field.name.lower()
            dtype = field.dataType.simpleString().lower()

            # --- ID Columns (should be string/varchar)
            if "_id" in colname:
                if "string" not in dtype:
                    results.append((table_name, colname, dtype, "❌ Expected STRING"))
                else:
                    results.append((table_name, colname, dtype, "✅ OK"))

            # --- Date/Datetime Columns
            elif any(x in colname for x in ["_date", "timestamp", "_at"]):
                if "timestamp" not in dtype and "date" not in dtype:
                    results.append((table_name, colname, dtype, "❌ Expected TIMESTAMP/DATE"))
                else:
                    results.append((table_name, colname, dtype, "✅ OK"))

            # --- Numeric Measures (price, value, score, sequential)
            elif any(x in colname for x in ["score", "value", "price", "sequential", "cost"]):
                if not any(x in dtype for x in ["int", "bigint", "float", "double", "decimal"]):
                    results.append((table_name, colname, dtype, "❌ Expected NUMERIC"))
                else:
                    results.append((table_name, colname, dtype, "✅ OK"))
    
    return results

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 6, Finished, Available, Finished)

In [5]:
results = validate_column_types(silver_tables)

datatype_table = PrettyTable(["Table", "Column", "Detected Type", "Validation"])
for row in results:
    datatype_table.add_row(row)

print(datatype_table)

StatementMeta(, dd40e63f-1591-4cde-a985-4098ad9b42d0, 7, Finished, Available, Finished)

+-----------+-------------------------------+---------------+--------------------+
|   Table   |             Column            | Detected Type |     Validation     |
+-----------+-------------------------------+---------------+--------------------+
|   orders  |            order_id           |     string    |       ✅ OK        |
|   orders  |          customer_id          |     string    |       ✅ OK        |
|   orders  |    order_purchase_timestamp   |   timestamp   |       ✅ OK        |
|   orders  |       order_approved_at       |   timestamp   |       ✅ OK        |
|   orders  |  order_delivered_carrier_date |   timestamp   |       ✅ OK        |
|   orders  | order_delivered_customer_date |   timestamp   |       ✅ OK        |
|   orders  | order_estimated_delivery_date |   timestamp   |       ✅ OK        |
|   items   |            order_id           |     string    |       ✅ OK        |
|   items   |         order_item_id         |     bigint    | ❌ Expected STRING |
|   items   |

##### As order_item_id is a count of items within a single order, it is expected that it is remains as an integer. 

2. Primary Key Uniqueness & Null Values Check

In [5]:
# Orders unique
orders.groupBy("order_id").count().filter("count > 1").show()

# Customers unique
customers.groupBy("customer_id").count().filter("count > 1").show()

# Sellers unique
sellers.groupBy("seller_id").count().filter("count > 1").show()

# Products unique
products.groupBy("product_id").count().filter("count > 1").show()

# Reviews unique
reviews.groupBy("review_id").count().filter("count > 1").show(5)
reviews.groupBy("review_id").count().filter("count > 1").agg(F.sum("count").alias("total_duplicate_rows")).show()

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 7, Finished, Available, Finished)

+--------+-----+
|order_id|count|
+--------+-----+
+--------+-----+

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+

+---------+-----+
|seller_id|count|
+---------+-----+
+---------+-----+

+----------+-----+
|product_id|count|
+----------+-----+
+----------+-----+

+--------------------+-----+
|           review_id|count|
+--------------------+-----+
|e4f5fbbbf2fa8f259...|    2|
|dbcd301b59c7e85e6...|    2|
|4812973f7fb59a488...|    2|
|f45e3373e177f5694...|    2|
|89d6214895235bb95...|    2|
+--------------------+-----+
only showing top 5 rows

+--------------------+
|total_duplicate_rows|
+--------------------+
|                1603|
+--------------------+



##### Let's investigate duplicate review_id records, and check whether the duplication matches how many items were in that order by looking at order_item_id in the items table.

In [47]:
# Join with reviews and items
duplicates_with_items = (
    reviews.join(
        duplicate_review_ids, "review_id", "inner"  # keep only duplicate reviews
    )
    .join(
        items.select("order_id", "order_item_id"),
        "order_id",
        "left"  # LEFT join to keep all reviews even if no matching items
    )
    .select(
        "review_id",
        "order_id",
        "review_comment_message",
        "order_item_id"
    )
)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 49, Finished, Available, Finished)

In [48]:
correspondence_check = (
    duplicates_with_items.groupBy("review_id", "order_id")
                          .agg(
                              F.countDistinct("order_item_id").alias("item_count"),
                              F.count("*").alias("review_count"),
                              F.collect_list("order_item_id").alias("item_list")
                          )
                          .withColumn("match_items", F.col("item_count") == F.col("review_count"))
)

correspondence_check.show(truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 50, Finished, Available, Finished)

+--------------------------------+--------------------------------+----------+------------+---------+-----------+
|review_id                       |order_id                        |item_count|review_count|item_list|match_items|
+--------------------------------+--------------------------------+----------+------------+---------+-----------+
|00130cbe1f9d422698c812ed8ded1919|04a28263e085d399c97ae49e0b477efa|1         |1           |[1]      |true       |
|00130cbe1f9d422698c812ed8ded1919|dfcdfc43867d1c1381bfaf62d6b9c195|1         |1           |[1]      |true       |
|0115633a9c298b6a98bcbe4eee75345f|0c9850b2c179c1ef60d2855e2751d1fa|2         |2           |[1, 2]   |true       |
|0115633a9c298b6a98bcbe4eee75345f|78a4201f58af3463bdab842eea4bc801|1         |1           |[1]      |true       |
|0174caf0ee5964646040cd94e15ac95e|74db91e33b4e1fd865356c89a61abf1f|1         |1           |[1]      |true       |
|0174caf0ee5964646040cd94e15ac95e|f93a732712407c02dce5dd5088d0f47b|1         |1         

In [61]:
duplicate_review_ids_df = (
    reviews.groupBy("review_id")
           .count()
           .filter(F.col("count") > 1)
           .select("review_id")
)

duplicate_review_ids = [row["review_id"] for row in duplicate_review_ids_df.collect()]

reviews.filter(F.col("review_id").isin(duplicate_review_ids)) \
       .orderBy("review_id", "order_id") \
       .show(truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 63, Finished, Available, Finished)

+--------------------------------+--------------------------------+------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------+
|review_id                       |order_id                        |review_score|review_comment_title|review_comment_message                                                                                                                                                                    |review_creation_date|review_answer_timestamp|
+--------------------------------+--------------------------------+------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------------

##### We found cases where the same review ID maps to multiple order IDs. This suggests either data duplication or data entry inconsistencies.

##### Upon reviewing the messages, they are identical with identical review creation date and time. Hence, this points to a duplication problem. We will need to investigate on this further by joining to the orders table.

##### Meanwhile, let's look into the cases of why there are review_ids for orders not found in the items table.

In [49]:
mismatched_items = correspondence_check.filter(F.col("match_items") == False)
mismatched_items.show(truncate=False)
print(f"Number of mismatched orders: {mismatched_items.count()}")

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 51, Finished, Available, Finished)

+--------------------------------+--------------------------------+----------+------------+---------+-----------+
|review_id                       |order_id                        |item_count|review_count|item_list|match_items|
+--------------------------------+--------------------------------+----------+------------+---------+-----------+
|0288d42bef3dfe36930740c9588a570f|33d8795f04dd631f3480d7aaf90da3dc|0         |1           |[]       |false      |
|0288d42bef3dfe36930740c9588a570f|f889a5a0b44adc29c5465b99395ac3c1|0         |1           |[]       |false      |
|047fd109ced39e02296f6aeb74f6a6f1|236e6ec6171c1870d4bcf4ccfad87f49|0         |1           |[]       |false      |
|047fd109ced39e02296f6aeb74f6a6f1|a89abace0dcc01eeb267a9660b5ac126|0         |1           |[]       |false      |
|08528f70f579f0c830189efc523d2182|03310aa823a66056268a3bab36e827fb|0         |1           |[]       |false      |
|08528f70f579f0c830189efc523d2182|7813842ae95e8c497fc0233232ae815a|0         |1         

In [51]:
# Check which ones have missing order_items
mismatched_items = (
    correspondence_check
    .withColumn(
        "mismatch_reason",
        F.when(F.col("item_count") == 0, "No matching items")
         .when(F.col("item_count") != F.col("review_count"), "Mismatch in review count vs items")
         .otherwise("Unknown reason")
    )
    .filter(F.col("match_items") == False)
)

mismatched_items.groupBy("mismatch_reason").count().show(truncate=False)
mismatched_items.show(truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 53, Finished, Available, Finished)

+-----------------+-----+
|mismatch_reason  |count|
+-----------------+-----+
|No matching items|63   |
+-----------------+-----+

+--------------------------------+--------------------------------+----------+------------+---------+-----------+-----------------+
|review_id                       |order_id                        |item_count|review_count|item_list|match_items|mismatch_reason  |
+--------------------------------+--------------------------------+----------+------------+---------+-----------+-----------------+
|0288d42bef3dfe36930740c9588a570f|33d8795f04dd631f3480d7aaf90da3dc|0         |1           |[]       |false      |No matching items|
|0288d42bef3dfe36930740c9588a570f|f889a5a0b44adc29c5465b99395ac3c1|0         |1           |[]       |false      |No matching items|
|047fd109ced39e02296f6aeb74f6a6f1|236e6ec6171c1870d4bcf4ccfad87f49|0         |1           |[]       |false      |No matching items|
|047fd109ced39e02296f6aeb74f6a6f1|a89abace0dcc01eeb267a9660b5ac126|0         

In [53]:
# Get order_ids with no matching items
no_item_orders = mismatched_items.filter(F.col("mismatch_reason") == "No matching items") \
                                  .select("order_id") \
                                  .distinct()

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 55, Finished, Available, Finished)

In [58]:
# Join with orders to check status
order_status_check = no_item_orders.join(
    orders.select("order_id", "order_status"),
    "order_id",
    "left"
)

order_status_check.groupBy("order_status").count().show(truncate=False)
order_status_check.show(truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 60, Finished, Available, Finished)

+------------+-----+
|order_status|count|
+------------+-----+
|canceled    |52   |
|unavailable |10   |
+------------+-----+

+--------------------------------+------------+
|order_id                        |order_status|
+--------------------------------+------------+
|0e6dbbc7484dc1ee0392c9ca801035c2|canceled    |
|f81e71dd7c044a9ee7c1d6e37d68d64f|canceled    |
|2663ce3526465b7ee9809428adabdc45|canceled    |
|74e198a81e758ec637a42e8f2e856927|canceled    |
|a89abace0dcc01eeb267a9660b5ac126|canceled    |
|5290c34bd38a8a095b885f13958db1e1|canceled    |
|b34369ee8397436dd7ceb8bb7783671d|canceled    |
|582fd4fa596ee1af88f153a5ba0523f2|canceled    |
|21a00b08cbeb5716bbb66105e3dbd850|canceled    |
|0d3adebce4bebc1f80a7f36e9833f497|canceled    |
|c186f4babae2b9ffa3ecc0176980acdf|unavailable |
|151e44f3dd461fe0838b6f59f884749f|unavailable |
|ed3efbd3a87bea76c2812c66a0b32219|canceled    |
|63802c52ced9ff923c962ecbf4b3a2a2|canceled    |
|04a2ba381317648726f46210a46ece95|canceled    |
|d2bc174d

##### There are 62 orders with reviews that are canceled or unavailable. It would be correct that they cannot be found in order_items. However, reviews with missing order items is 1.

##### Let's also check the review messages for these canceled or unavailable orders.

In [59]:
# Orders with no matching items
no_item_orders = mismatched_items.filter(F.col("mismatch_reason") == "No matching items") \
                                  .select("order_id") \
                                  .distinct()
# Join orders with reviews
no_item_order_reviews = no_item_orders.join(
    orders.select("order_id", "order_status"),
    "order_id",
    "left"
).join(
    reviews.select("order_id", "review_id", "review_comment_message"),
    "order_id",
    "left"
).select(
    "order_id",
    "order_status",
    "review_id",
    "review_comment_message"
)

# Count null messages
no_item_order_reviews.filter(F.col("review_comment_message").isNull()) \
                     .groupBy("order_status") \
                     .count() \
                     .show(truncate=False)

no_item_order_reviews.show(50, truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 61, Finished, Available, Finished)

+------------+-----+
|order_status|count|
+------------+-----+
|canceled    |19   |
|unavailable |8    |
+------------+-----+

+--------------------------------+------------+--------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id                        |order_status|review_id                       |review_comment_message                                                                                                                                                                                 |
+--------------------------------+------------+--------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0e6dbbc7484dc1ee0392c9ca80103

##### From reviewing the messages from the 62 canceled and unavailable records, there are 27 nulls. 

##### For messages that are not null, it seems that some of them may have been refunded as the messages point at wrongly delivered items. Since there is no return category, it could have been marked as canceled instead. We will need to highlight to the Olist team that adding a return or refund category will give clarity for analysis. 

##### For the Gold lakehouse, let's add in the order_status to the fact_reviews table for ease of analysis. This would enable filtering of the mismatched items.

3. Foreign Key Validation

In [4]:
# Order items → Orders: orphaned records in items that don’t have a corresponding order in orders. 
items.join(orders, "order_id", "left_anti").show()    #8 orders missing from orders table 

# Payments → Orders: orphaned records in payments that don’t have a corresponding order in orders. 
payments.join(orders, "order_id", "left_anti").show()    #8 orders missing from orders table

# Reviews → Orders : orphaned records in reviews that don’t have a corresponding order in orders. 
reviews.join(orders, "order_id", "left_anti").show()    #8 orders missing from orders table

# Products → Order Items
items.join(products, "product_id", "left_anti").show()

# Sellers → Order Items
items.join(sellers, "seller_id", "left_anti").show()

StatementMeta(, cff59718-88ef-4786-aea3-6b4be8b8f028, 6, Finished, Available, Finished)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|ab7c89dc1bf4a1ead...|            1|a2a7efc985315e86d...|ed4acab38528488b6...|2018-06-18 12:30:35|110.99|         9.13|
|0d3268bad9b086af7...|            1|ec165cd31c5058578...|8bdd8e3fd58bafa48...|2018-07-05 21:29:54|188.99|        15.63|
|e69f75a717d64fc5e...|            1|e7d5464b94c9a5963...|58f1a6197ed863543...|2018-07-05 22:15:14| 139.0|        19.07|
|f5dd62b788049ad9f...|            1|2167c8f6252667c0e...|0bb738e4d789e63e2...|2018-06-26 07:19:05| 329.0|        25.24|
|20edc82cf5400ce95...|            1|55bfa0307d7a46bed...|343e716476e3748b0...|2018-07-03 16:29:30|  45.9|         9.07|
|2d1e2d5bf4dc7227b...|            1|a50a

Let's check if the 8 records missing in order but found in items, payments and reviews are the same.

In [5]:
unmatched_reviews = reviews.join(orders, "order_id", "left_anti").select("order_id")
unmatched_payments = payments.join(orders, "order_id", "left_anti").select("order_id")
unmatched_items = items.join(orders, "order_id", "left_anti").select("order_id")

StatementMeta(, cff59718-88ef-4786-aea3-6b4be8b8f028, 7, Finished, Available, Finished)

In [6]:
common_unmatched = (
    unmatched_reviews
    .intersect(unmatched_payments)
    .intersect(unmatched_items)
)
common_unmatched.show()

StatementMeta(, cff59718-88ef-4786-aea3-6b4be8b8f028, 8, Finished, Available, Finished)

+--------------------+
|            order_id|
+--------------------+
|e69f75a717d64fc5e...|
|0d3268bad9b086af7...|
|2ebdfc4f15f23b914...|
|2d858f451373b04fb...|
|2d1e2d5bf4dc7227b...|
|f5dd62b788049ad9f...|
|ab7c89dc1bf4a1ead...|
|20edc82cf5400ce95...|
+--------------------+



In [10]:
# Add these order_ids into the orders table in silver lakehouse with other column values as null
table_columns = ["order_id", "customer_id", "order_status", "order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"]

# Add missing columns with null values
for col in table_columns:
    if col not in common_unmatched.columns:
        common_unmatched = common_unmatched.withColumn(col, lit(None))

# Reorder columns to match the table schema
common_unmatched = common_unmatched.select(table_columns)

# Append to the Delta table in silver lakehouse
common_unmatched.write.format("delta").mode("append").saveAsTable("olist_orders_cleaned")

StatementMeta(, cff59718-88ef-4786-aea3-6b4be8b8f028, 12, Finished, Available, Finished)

##### Yes, the 8 records missing from the orders table are found in the reviews, payments and order_items tables. 

##### Since, they are not in the orders table, we are unable to check on the order_status. 

##### Hence, we can assume that they are valid orders that were successful (delivered). 

##### We will highlight this to the Olist Team to check on inconsistency of data flow from the system. but meanwhile, in order to populate our Gold lakehouse table, we will have to insert these 8 orders to the orders table. 

4. Null Values and Outliers

In [7]:
# Check for nulls in key columns
def check_nulls(df, colname):
    return df.filter(col(colname).isNull() | isnan(col(colname))).count()

print("Null orders:", check_nulls(orders, "order_id"))
print("Null customers:", check_nulls(customers, "customer_id"))
print("Null products:", check_nulls(products, "product_id"))
print("Null sellers:", check_nulls(sellers, "seller_id"))

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 9, Finished, Available, Finished)

Null orders: 0
Null customers: 0
Null products: 0
Null sellers: 0


In [8]:
# Negative/Zero prices or freight
items.filter((col("price") <= 0) | (col("freight_value") < 0)).show()

# Review scores out of range
reviews.filter((col("review_score") < 1) | (col("review_score") > 5)).show()

# Payment installments invalid
payments.filter(col("payment_installments") < 1).show() 

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 10, Finished, Available, Finished)

+--------+-------------+----------+---------+-------------------+-----+-------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|
+--------+-------------+----------+---------+-------------------+-----+-------------+
+--------+-------------+----------+---------+-------------------+-----+-------------+

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+

+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+-----

5. Data Consistency Check

In [6]:
spark.conf.get("spark.sql.session.timeZone")

StatementMeta(, 9873af4d-6a72-4731-8650-5ef8218963c7, 8, Finished, Available, Finished)

'UTC'

In [5]:
# Orders: check timestamp order

orders_fixed = orders
for c in [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]:
    orders_fixed = orders_fixed.withColumn(c, to_timestamp(col(c)))

orders_anomaly = orders_fixed.filter(
    ~(
        (col("order_purchase_timestamp") <= col("order_approved_at")) &
        (col("order_approved_at") <= col("order_delivered_carrier_date"))
    )
).select(
    "order_id", "order_purchase_timestamp", "order_approved_at",
    "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"
)

print(f"Total orders with timestamp anomaly: {orders_anomaly.count()}")
orders_anomaly.show()

StatementMeta(, 9873af4d-6a72-4731-8650-5ef8218963c7, 7, Finished, Available, Finished)

Total orders with timestamp anomaly: 1359
+--------------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|fb6343faebbbee8c2...|     2018-07-04 11:27:08|2018-07-05 16:33:04|         2018-07-05 15:51:00|          2018-07-11 19:26:27|          2018-08-06 00:00:00|
|721b6eed5d18982b6...|     2018-04-21 10:13:19|2018-04-24 17:27:06|         2018-04-23 18:22:02|          2018-04-26 03:12:03|          2018-05-08 00:00:00|
|478b98b4c7442950e...|     2018-04-21 00:33:15|2018-04-24 19:27:57|         2018-04-24 18:19:48|          2018-04-30 21:57:49|          2018-05-18 00:00:00|
|fba03bf5e3db258

In [10]:
# Reviews: check sequence vs orders
reviews_timestamp = reviews.join(orders, "order_id", "inner").filter(
    (col("review_creation_date") < col("order_purchase_timestamp")) |
    (col("review_answer_timestamp") < col("review_creation_date"))
).select("review_id", "order_id", "order_purchase_timestamp","review_creation_date", "review_answer_timestamp")

print(f"Total reviews with timestamp anomaly: {reviews_timestamp.count()}")
reviews_timestamp.show(5)


StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 12, Finished, Available, Finished)

Total reviews with timestamp anomaly: 74
+--------------------+--------------------+------------------------+--------------------+-----------------------+
|           review_id|            order_id|order_purchase_timestamp|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------------------+--------------------+-----------------------+
|58f1655df206a9a40...|f81e71dd7c044a9ee...|     2018-08-15 08:34:42| 2018-07-21 00:00:00|    2018-07-24 00:09:40|
|047fd109ced39e022...|a89abace0dcc01eeb...|     2018-09-06 18:45:47| 2018-07-31 00:00:00|    2018-08-02 15:11:49|
|5849ab48269292920...|b102b42cde7719ae4...|     2018-08-29 10:15:27| 2018-08-12 00:00:00|    2018-08-12 14:44:26|
|59c45120064c0bcfd...|afed0d88ec7753e7b...|     2018-08-02 15:23:07| 2018-07-28 00:00:00|    2018-07-29 11:14:13|
|f45e3373e177f5694...|2663ce3526465b7ee...|     2018-08-30 13:07:46| 2018-08-23 00:00:00|    2018-08-24 00:31:45|
+--------------------+--------------------+----

Upon inpection, it seems that anomaly rows have the correct logic and inherently there is something deeper that cannot be identified for the orders table. 

For the reviews table, it does seem that review creation date was before the order_purchase_timestamp. This issue will be flagged to the downstream teams and to the developers of the Olist app to check closely on how the timestamps are captured for data collection.

6. Business Rules Check

In [30]:
# Check if price + frieght value in items match payment value in payments.

# 1) per-item total (handle nulls and ensure numeric types)
items_with_item_total = items.withColumn(
    "item_total",
    (F.coalesce(F.col("price").cast("double"), F.lit(0.0))
     + F.coalesce(F.col("freight_value").cast("double"), F.lit(0.0)))
)

# 2) order-level total by summing item_total
order_total_df = (
    items_with_item_total
    .groupBy("order_id")
    .agg(F.sum("item_total").cast("double").alias("order_total"))
)

# 3) payments total per order (sum of installments)
payments_total_df = (
    payments
    .groupBy("order_id")
    .agg(F.sum(F.coalesce(F.col("payment_value").cast("double"), F.lit(0.0))).alias("total_payments"))
)

# 4) join order-level totals + payments back to items so you can inspect per order_item
payment_check = (
    items_with_item_total
    .join(order_total_df, "order_id", "left")
    .join(payments_total_df, "order_id", "left")
    .select(
        "order_id",
        "order_item_id",
        "item_total",
        "order_total",
        "total_payments"
    )
    .withColumn(
        "diff",
        F.round(
            F.coalesce(F.col("total_payments"), F.lit(0.0)) -
            F.coalesce(F.col("order_total"), F.lit(0.0)),
            2
        )
    )
    .withColumn(
        "diff_percent",
        F.round(
            (F.col("diff") / F.coalesce(F.col("total_payments"), F.lit(1.0))) * 100,
            2
        )
    )
)

# 5) filter mismatches (tolerance = 1) and show results + count
mismatches = payment_check.filter(F.abs(F.col("diff")) > 1)

print(f"Total orders with mismatch (abs(diff) > 1): {mismatches.select('order_id').distinct().count()}")
mismatches.select("order_id", "order_item_id", "item_total", "order_total", "total_payments", "diff", "diff_percent").orderBy("diff_percent", ascending=False).show(50, truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 32, Finished, Available, Finished)

Total orders with mismatch (abs(diff) > 1): 250
+--------------------------------+-------------+------------------+------------------+-----------------+------+------------+
|order_id                        |order_item_id|item_total        |order_total       |total_payments   |diff  |diff_percent|
+--------------------------------+-------------+------------------+------------------+-----------------+------+------------+
|6e5fe7366a2e1bfbf3257dba0af1267f|1            |36.83             |287.90999999999997|406.92           |119.01|29.25       |
|6e5fe7366a2e1bfbf3257dba0af1267f|2            |59.14             |287.90999999999997|406.92           |119.01|29.25       |
|6e5fe7366a2e1bfbf3257dba0af1267f|3            |36.83             |287.90999999999997|406.92           |119.01|29.25       |
|6e5fe7366a2e1bfbf3257dba0af1267f|4            |59.14             |287.90999999999997|406.92           |119.01|29.25       |
|6e5fe7366a2e1bfbf3257dba0af1267f|5            |36.83             |287.909999

There is no definitive answer as to why customers paid more or less for these 250 orders as we do not have more information.

For orders that customer have paid more, there is no conclusive reason.

For orders where customers paid less, there could have been a voucher use or redemption (for cases where there was no payment at all). However, we do not have these information. 

We will assume that payment value is the final payment made by the customer and collected by Olist.

In [13]:
# Orders (not canceled) that have no item records — i.e., incomplete or corrupted orders
missing_orders = (orders.filter(F.col("order_status") != "canceled").join(items, "order_id", "left_anti"))

print(f"Total missing (non-cancelled) orders: {missing_orders.count()}")
missing_orders.show(5)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 15, Finished, Available, Finished)

Total missing (non-cancelled) orders: 611
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|6338011716bfe53b9...|74671791c2cc61942...| unavailable|     2017-11-16 12:28:25|2017-11-16 12:56:02|                        NULL|                         NULL|          2017-11-29 00:00:00|
|90e258ad907d8401c...|358a7b26ad67e3a18...| unavailable|     2017-09-25 16:03:27|2017-09-25 16:15:19|                        NULL|                         NULL|          2017-10-31 00:00:00|
|4f

In [20]:
# Check if these missing orders have payments
payment_check = missing_orders.join(payments, "order_id", "left")

print(f"Total unavailable order status with payments: {payment_check.count()}")
payment_check.select("order_id", "order_status", "order_purchase_timestamp", "order_delivered_carrier_date", "payment_value").show(5, truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 22, Finished, Available, Finished)

Total unavailable order status with payments: 651
+--------------------------------+------------+------------------------+----------------------------+-------------+
|order_id                        |order_status|order_purchase_timestamp|order_delivered_carrier_date|payment_value|
+--------------------------------+------------+------------------------+----------------------------+-------------+
|6338011716bfe53b960847be47479662|unavailable |2017-11-16 12:28:25     |NULL                        |61.19        |
|90e258ad907d8401c4f1107ea9e55f1b|unavailable |2017-09-25 16:03:27     |NULL                        |130.91       |
|4ffd41fadb59bd398d869be78f1b2be3|unavailable |2018-03-26 09:37:59     |NULL                        |789.55       |
|62492d216bd7ca39cab1f24109b6bfd7|unavailable |2017-11-05 08:26:50     |NULL                        |215.94       |
|2636a02ee7de9590df86a4c24b739c49|unavailable |2017-06-29 09:10:37     |NULL                        |142.09       |
+---------------------

Since these orders with the "unavailable" status do not have an order_delivered_carrier_date but has a payment value, we are unable to confirm if these orders were later refunded due to failed delivery. 

Hence, we will need to caution the data analysts and data scientists to exclude these from any revenue calculations.

But they will still be available in the gold lakehouse orders table for any logistics related queries. 

In [24]:
# All non-canceled orders that do not have any payment record in the payments table.
missing_payments = (orders.filter(F.col("order_status") != "canceled").join(payments, "order_id", "left_anti"))

print(f"Total missing (non-cancelled) payments: {missing_payments.count()}")
missing_payments.select("order_id", "order_status", "order_delivered_carrier_date").show(5, truncate=False)

StatementMeta(, ca5e5fad-48f6-45e4-82ef-3451698b7ffc, 26, Finished, Available, Finished)

Total missing (non-cancelled) payments: 1
+--------------------------------+------------+----------------------------+
|order_id                        |order_status|order_delivered_carrier_date|
+--------------------------------+------------+----------------------------+
|bfbd0f9bdef84302105ad712db648a6c|delivered   |2016-11-07 17:11:53         |
+--------------------------------+------------+----------------------------+



Since only one order is affected, we can conclude that there could have been some kind of error that has caused this order to be missed in the payment records.

This will be highlighted to the data analysts as well and the error margin would be low even if the amount is omitted. 